# Image classification

Image classification assigns a label or class to an image. Unlike text or audio classification, the inputs are the
pixel values that comprise an image. There are many applications for image classification, such as detecting damage
after a natural disaster, monitoring crop health, or helping screen medical images for signs of disease.

This guide illustrates how to:

1. Fine-tune [ViT](https://huggingface.co/docs/transformers/main/en/tasks/model_doc/vit) on the Alzheimer dataset to classify a food item in an image.
2. Use your fine-tuned model for inference.

<Tip>
The task illustrated in this tutorial is supported by the following model architectures:

<!--This tip is automatically generated by `make fix-copies`, do not fill manually!-->

[BEiT](https://huggingface.co/docs/transformers/main/en/tasks/../model_doc/beit), [BiT](https://huggingface.co/docs/transformers/main/en/tasks/../model_doc/bit), [ConvNeXT](https://huggingface.co/docs/transformers/main/en/tasks/../model_doc/convnext), [ConvNeXTV2](https://huggingface.co/docs/transformers/main/en/tasks/../model_doc/convnextv2), [CvT](https://huggingface.co/docs/transformers/main/en/tasks/../model_doc/cvt), [Data2VecVision](https://huggingface.co/docs/transformers/main/en/tasks/../model_doc/data2vec-vision), [DeiT](https://huggingface.co/docs/transformers/main/en/tasks/../model_doc/deit), [DiNAT](https://huggingface.co/docs/transformers/main/en/tasks/../model_doc/dinat), [EfficientFormer](https://huggingface.co/docs/transformers/main/en/tasks/../model_doc/efficientformer), [EfficientNet](https://huggingface.co/docs/transformers/main/en/tasks/../model_doc/efficientnet), [FocalNet](https://huggingface.co/docs/transformers/main/en/tasks/../model_doc/focalnet), [ImageGPT](https://huggingface.co/docs/transformers/main/en/tasks/../model_doc/imagegpt), [LeViT](https://huggingface.co/docs/transformers/main/en/tasks/../model_doc/levit), [MobileNetV1](https://huggingface.co/docs/transformers/main/en/tasks/../model_doc/mobilenet_v1), [MobileNetV2](https://huggingface.co/docs/transformers/main/en/tasks/../model_doc/mobilenet_v2), [MobileViT](https://huggingface.co/docs/transformers/main/en/tasks/../model_doc/mobilevit), [MobileViTV2](https://huggingface.co/docs/transformers/main/en/tasks/../model_doc/mobilevitv2), [NAT](https://huggingface.co/docs/transformers/main/en/tasks/../model_doc/nat), [Perceiver](https://huggingface.co/docs/transformers/main/en/tasks/../model_doc/perceiver), [PoolFormer](https://huggingface.co/docs/transformers/main/en/tasks/../model_doc/poolformer), [RegNet](https://huggingface.co/docs/transformers/main/en/tasks/../model_doc/regnet), [ResNet](https://huggingface.co/docs/transformers/main/en/tasks/../model_doc/resnet), [SegFormer](https://huggingface.co/docs/transformers/main/en/tasks/../model_doc/segformer), [SwiftFormer](https://huggingface.co/docs/transformers/main/en/tasks/../model_doc/swiftformer), [Swin Transformer](https://huggingface.co/docs/transformers/main/en/tasks/../model_doc/swin), [Swin Transformer V2](https://huggingface.co/docs/transformers/main/en/tasks/../model_doc/swinv2), [VAN](https://huggingface.co/docs/transformers/main/en/tasks/../model_doc/van), [ViT](https://huggingface.co/docs/transformers/main/en/tasks/../model_doc/vit), [ViT Hybrid](https://huggingface.co/docs/transformers/main/en/tasks/../model_doc/vit_hybrid), [ViTMSN](https://huggingface.co/docs/transformers/main/en/tasks/../model_doc/vit_msn)
<!--End of the generated tip-->

</Tip>

Before you begin, make sure you have all the necessary libraries installed:

```bash
pip install transformers datasets evaluate
```

We encourage you to log in to your Hugging Face account to upload and share your model with the community. When prompted, enter your token to log in:

## Load Alzheimer dataset

In [1]:
model_checkpoint = "google/vit-base-patch16-224-in21k"  # pre-trained model from which to fine-tune

In [2]:
from datasets import load_dataset

In [3]:
data = load_dataset('imagefolder', data_dir="new")

Resolving data files:   0%|          | 0/12719 [00:00<?, ?it/s]

In [4]:
data

DatasetDict({
    train: Dataset({
        features: ['image', 'label'],
        num_rows: 12719
    })
})

In [5]:
data = data['train'].train_test_split(test_size = 0.1, shuffle = True)
train_ds = data['train']
val_ds = data['test']

In [6]:
train_ds

Dataset({
    features: ['image', 'label'],
    num_rows: 11447
})

In [7]:
val_ds

Dataset({
    features: ['image', 'label'],
    num_rows: 1272
})

In [8]:
test_ds = load_dataset('imagefolder', data_dir='Benchmark/')['train']

Resolving data files:   0%|          | 0/600 [00:00<?, ?it/s]

In [9]:
train_ds[0]

{'image': <PIL.JpegImagePlugin.JpegImageFile image mode=L size=128x128>,
 'label': 3}

In [10]:
labels = data['train'].features["label"].names
label2id, id2label = dict(), dict()
for i, label in enumerate(labels):
    label2id[label] = str(i)
    id2label[str(i)] = label

label2id

{'Mild_Demented': '0',
 'Moderate_Demented': '1',
 'Non_Demented': '2',
 'Very_Mild_Demented': '3'}

In [11]:
from transformers import AutoImageProcessor

checkpoint = "google/vit-base-patch16-224-in21k"
image_processor = AutoImageProcessor.from_pretrained(checkpoint)

In [12]:
from torchvision.transforms import RandomResizedCrop, Compose, Normalize, ToTensor

normalize = Normalize(mean=image_processor.image_mean, std=image_processor.image_std)
size = (
    image_processor.size["shortest_edge"]
    if "shortest_edge" in image_processor.size
    else (image_processor.size["height"], image_processor.size["width"])
)
_transforms = Compose([RandomResizedCrop(size), ToTensor(), normalize])

In [13]:
def transforms(examples):
    examples["pixel_values"] = [_transforms(img.convert("RGB")) for img in examples["image"]]
    del examples["image"]
    return examples

In [14]:
train_ds = train_ds.with_transform(transforms)
val_ds = val_ds.with_transform(transforms)
test_ds = test_ds.with_transform(transforms)

In [15]:
from transformers import DefaultDataCollator

data_collator = DefaultDataCollator()

In [16]:
import evaluate

accuracy = evaluate.load("accuracy")

In [17]:
import numpy as np


def compute_metrics(eval_pred):
    predictions, labels = eval_pred
    predictions = np.argmax(predictions, axis=1)
    return accuracy.compute(predictions=predictions, references=labels)

In [18]:
from transformers import AutoModelForImageClassification, TrainingArguments, Trainer

model = AutoModelForImageClassification.from_pretrained(
    checkpoint,
    num_labels=len(labels),
    id2label=id2label,
    label2id=label2id,
)

Some weights of ViTForImageClassification were not initialized from the model checkpoint at google/vit-base-patch16-224-in21k and are newly initialized: ['classifier.bias', 'classifier.weight']
You should probably TRAIN this model on a down-stream task to be able to use it for predictions and inference.


In [19]:
training_args = TrainingArguments(
    output_dir="alzheimer_classification_base_google",
    f
    remove_unused_columns=False,
    evaluation_strategy="epoch",
    save_strategy="epoch",
    learning_rate=5e-5,
    per_device_train_batch_size=4,
    gradient_accumulation_steps=4,
    per_device_eval_batch_size=4,
    num_train_epochs=20,
    warmup_ratio=0.1,
    logging_steps=10,
    load_best_model_at_end=True,
    metric_for_best_model="accuracy",
    push_to_hub=False)

trainer = Trainer(
    model=model,
    args=training_args,
    data_collator=data_collator,
    train_dataset=train_ds,
    eval_dataset=val_ds,
    tokenizer=image_processor,
    compute_metrics=compute_metrics)

trainer.train()

c:\Users\User\anaconda3\envs\sic2\Lib\site-packages\transformers\optimization.py:411: FutureWarning: This implementation of AdamW is deprecated and will be removed in a future version. Use the PyTorch implementation torch.optim.AdamW instead, or set `no_deprecation_warning=True` to disable this warning
  warnings.warn(


  0%|          | 0/14300 [00:00<?, ?it/s]

{'loss': 1.3802, 'learning_rate': 3.496503496503497e-07, 'epoch': 0.01}
{'loss': 1.3687, 'learning_rate': 6.993006993006994e-07, 'epoch': 0.03}
{'loss': 1.3677, 'learning_rate': 1.0489510489510491e-06, 'epoch': 0.04}
{'loss': 1.3769, 'learning_rate': 1.3986013986013987e-06, 'epoch': 0.06}
{'loss': 1.3635, 'learning_rate': 1.7482517482517485e-06, 'epoch': 0.07}
{'loss': 1.3625, 'learning_rate': 2.0979020979020983e-06, 'epoch': 0.08}
{'loss': 1.3544, 'learning_rate': 2.4475524475524477e-06, 'epoch': 0.1}
{'loss': 1.3505, 'learning_rate': 2.7972027972027974e-06, 'epoch': 0.11}
{'loss': 1.315, 'learning_rate': 3.1468531468531472e-06, 'epoch': 0.13}
{'loss': 1.3074, 'learning_rate': 3.496503496503497e-06, 'epoch': 0.14}
{'loss': 1.2842, 'learning_rate': 3.846153846153847e-06, 'epoch': 0.15}
{'loss': 1.27, 'learning_rate': 4.195804195804197e-06, 'epoch': 0.17}
{'loss': 1.2687, 'learning_rate': 4.5454545454545455e-06, 'epoch': 0.18}
{'loss': 1.2154, 'learning_rate': 4.895104895104895e-06, 'ep

  0%|          | 0/318 [00:00<?, ?it/s]

{'eval_loss': 0.7527833580970764, 'eval_accuracy': 0.6729559748427673, 'eval_runtime': 11.1591, 'eval_samples_per_second': 113.987, 'eval_steps_per_second': 28.497, 'epoch': 1.0}
{'loss': 0.6848, 'learning_rate': 2.5174825174825178e-05, 'epoch': 1.01}
{'loss': 0.7226, 'learning_rate': 2.5524475524475528e-05, 'epoch': 1.02}
{'loss': 0.7636, 'learning_rate': 2.5874125874125877e-05, 'epoch': 1.03}
{'loss': 0.7563, 'learning_rate': 2.6223776223776224e-05, 'epoch': 1.05}
{'loss': 0.7575, 'learning_rate': 2.6573426573426574e-05, 'epoch': 1.06}
{'loss': 0.8642, 'learning_rate': 2.6923076923076923e-05, 'epoch': 1.08}
{'loss': 0.6693, 'learning_rate': 2.7272727272727273e-05, 'epoch': 1.09}
{'loss': 0.7255, 'learning_rate': 2.762237762237762e-05, 'epoch': 1.1}
{'loss': 0.7696, 'learning_rate': 2.7972027972027976e-05, 'epoch': 1.12}
{'loss': 0.6687, 'learning_rate': 2.8321678321678326e-05, 'epoch': 1.13}
{'loss': 0.701, 'learning_rate': 2.8671328671328672e-05, 'epoch': 1.15}
{'loss': 0.7259, 'lea

...

  0%|          | 0/318 [00:00<?, ?it/s]

{'eval_loss': 0.13294503092765808, 'eval_accuracy': 0.9536163522012578, 'eval_runtime': 16.782, 'eval_samples_per_second': 75.796, 'eval_steps_per_second': 18.949, 'epoch': 19.0}
{'loss': 0.1146, 'learning_rate': 2.7195027195027198e-06, 'epoch': 19.01}
{'loss': 0.0694, 'learning_rate': 2.6806526806526807e-06, 'epoch': 19.02}
{'loss': 0.0518, 'learning_rate': 2.641802641802642e-06, 'epoch': 19.04}
{'loss': 0.0559, 'learning_rate': 2.602952602952603e-06, 'epoch': 19.05}
{'loss': 0.0643, 'learning_rate': 2.564102564102564e-06, 'epoch': 19.06}
{'loss': 0.0594, 'learning_rate': 2.5252525252525253e-06, 'epoch': 19.08}
{'loss': 0.0649, 'learning_rate': 2.4864024864024867e-06, 'epoch': 19.09}
{'loss': 0.1332, 'learning_rate': 2.4475524475524477e-06, 'epoch': 19.11}
{'loss': 0.0664, 'learning_rate': 2.408702408702409e-06, 'epoch': 19.12}
{'loss': 0.1164, 'learning_rate': 2.36985236985237e-06, 'epoch': 19.13}
{'loss': 0.0438, 'learning_rate': 2.3310023310023313e-06, 'epoch': 19.15}
{'loss': 0.07

  0%|          | 0/318 [00:00<?, ?it/s]

{'eval_loss': 0.1428692489862442, 'eval_accuracy': 0.9520440251572327, 'eval_runtime': 16.6446, 'eval_samples_per_second': 76.421, 'eval_steps_per_second': 19.105, 'epoch': 19.99}
{'train_runtime': 7944.0985, 'train_samples_per_second': 28.819, 'train_steps_per_second': 1.8, 'train_loss': 0.29136589384683365, 'epoch': 19.99}


TrainOutput(global_step=14300, training_loss=0.29136589384683365, metrics={'train_runtime': 7944.0985, 'train_samples_per_second': 28.819, 'train_steps_per_second': 1.8, 'train_loss': 0.29136589384683365, 'epoch': 19.99})

In [28]:
trainer.push_to_hub("alzheimer-classification-google-base")

OSError: Tried to clone a repository in a non-empty folder that isn't a git repository ('d:\Desktop\SIC Project\datafolder\AugmentedAlzheimerDataset\alzheimer_classification_base_google'). If you really want to do this, do it manually:
 cd d:\Desktop\SIC Project\datafolder\AugmentedAlzheimerDataset\alzheimer_classification_base_google && git init && git remote add origin && git pull origin main
 or clone repo to a new folder and move your existing files there afterwards.